In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

In [2]:
def gen_rand_lat_long():
    lat = (random.uniform(12.7910, 13.0192))
    long = (random.uniform(77.5490, 77.6940))
    return lat,long


In [15]:


num_customers = 100

start_date = datetime.today()

departure_times = {
    0: [(7, 0), (8, 30)],  # Monday: Either 7 AM or 8:30 AM
    1: [(7, 30), (9, 0)],  # Tuesday: Either 7:30 AM or 9:00 AM
    2: [(7, 0), (8, 0)],   # Wednesday: Either 7 AM or 8 AM
    3: [(8, 0), (9, 30)],  # Thursday: Either 8 AM or 9:30 AM
    4: [(7, 30), (8, 30)],  # Friday: Either 7:30 AM or 8:30 AM
    5: [(8, 30), (10, 0)],  # Saturday: Either 8:30 AM or 10:00 AM
    6: [(9, 0), (10, 30)],  # Sunday: Either 9 AM or 10:30 AM
}

def trip_rec():
    trip_data=[]
    for customer_id in range(1, num_customers + 1):
        home_lat, home_lon = gen_rand_lat_long()
        office_lat, office_lon = gen_rand_lat_long()

        for day in range(0, 90): 
            date = start_date + timedelta(days=day)
            departure_hour, departure_minute = random.choice(departure_times[date.weekday()])
            baseline_duration = np.random.uniform(30, 60) 
            traffic_factor = 1 + (np.random.uniform(0, 0.2))  
            trip_duration = baseline_duration * traffic_factor  
            for i in ['Home_to_office']:
                    trip_data.append({
                    'customer_id': customer_id,
                    'date': date.strftime('%Y-%m-%d'),
                    'departure_time': f'{departure_hour:02}:{departure_minute:02}',
                    'trip_type': i,
                    'home_lat': home_lat ,
                    'home_lon': home_lon ,
                    'office_lat': office_lat ,
                    'office_lon': office_lon ,
                    'distance':1,
                    'trip_duration': round(trip_duration, 2)  # Round to 2 decimal places
                })
    
    return trip_data

trip_records = trip_rec()

df_trips = pd.DataFrame(trip_records)

display(df_trips)
df_trips.to_csv("../JJ_ETA/Data/Sample.csv")

,customer_id,date,departure_time,trip_type,home_lat,home_lon,office_lat,office_lon,distance,trip_duration
0,1,2025-04-11,08:30,Home_to_office,12.909052,77.659128,12.899790,77.661176,1,55.44
1,1,2025-04-12,10:00,Home_to_office,12.909052,77.659128,12.899790,77.661176,1,35.36
2,1,2025-04-13,10:30,Home_to_office,12.909052,77.659128,12.899790,77.661176,1,34.80
3,1,2025-04-14,08:30,Home_to_office,12.909052,77.659128,12.899790,77.661176,1,36.43
4,1,2025-04-15,09:00,Home_to_office,12.909052,77.659128,12.899790,77.661176,1,43.70
...,...,...,...,...,...,...,...,...,...,...
8995,100,2025-07-05,10:00,Home_to_office,12.929443,77.691459,12.823473,77.657542,1,48.00
8996,100,2025-07-06,10:30,Home_to_office,12.929443,77.691459,12.823473,77.657542,1,54.05
8997,100,2025-07-07,07:00,Home_to_office,12.929443,77.691459,12.823473,77.657542,1,57.27
8998,100,2025-07-08,09:00,Home_to_office,12.929443,77.691459,12.823473,77.657542,1,61.32


In [16]:
import requests
def get_distance(lat1, lon1, lat2, lon2):
    # Construct the URL for the OSRM API
    url = f'http://router.project-osrm.org/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false'
    
    # Send GET request to the API
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Extract the distance from the response
        distance = data['routes'][0]['legs'][0]['distance']  
        return distance / 1000  
    else:
        return f"Error: Unable to get data (Status Code: {response.status_code})"


In [17]:
for i in range(df_trips.shape[0]):
    df_trips['distance'][i]=get_distance(df_trips['home_lat'][i], df_trips['home_lon'][i], df_trips['office_lon'][i], df_trips['office_lon'][i])
display(df_trips)

C:\Users\pratham\AppData\Local\Temp\ipykernel_14420\2743607708.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_trips['distance'][i]=get_distance(df_trips['home_lat'][i], df_trips['home_lon'][i], df_trips['office_lon'][i], df_trips['of

KeyboardInterrupt: 